<a href="https://colab.research.google.com/github/Nithya9404/Deep_Learning/blob/main/Deep_learning4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/fake.csv')
df.head(5)

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


In [ ]:
headlines = df['thread_title'].tolist()

In [ ]:
# Assuming headlines is a list of strings with some non-text values
headlines = [str(headline) for headline in headlines]
headlines = headlines[:20]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(headlines)
total_words = len(tokenizer.word_index) + 1


In [ ]:
len(headlines)

20

In [ ]:
headlines

['Muslims BUSTED: They Stole Millions In Gov’t Benefits',
 'Re: Why Did Attorney General Loretta Lynch Plead The Fifth?',
 'BREAKING: Weiner Cooperating With FBI On Hillary Email Investigation',
 'PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnapped And Killed By ISIS: "I have voted for Donald J. Trump!" » 100percentfedUp.com',
 "FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Healthcare Begins With A Bombshell! » 100percentfedUp.com",
 'Hillary Goes Absolutely Berserk On Protester At Rally! (Video)',
 'BREAKING! NYPD Ready To Make Arrests In Weiner Case…Hillary Visited Pedophile Island At Least 6 Times…Money Laundering, Underage Sex, Pay-for-Play,Proof of Inappropriate Handling Classified Information » 100percentfedUp.com',
 'WOW! WHISTLEBLOWER TELLS CHILLING STORY Of Massive Voter Fraud: Trump Campaign Readies Lawsuit Against FL Sec Of Elections In Critical District [VIDEO] » 100percentfedUp.com',
 "BREAKING: CLINTON CLEARED...Was This A Coordinated Last Minute Trick To Energize Hillary's Base?

In [ ]:
input_sequences = []
for line in headlines:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

In [ ]:
predictors, label = input_sequences[:, :-1], input_sequences[:, -1]

In [ ]:
label = tf.keras.utils.to_categorical(label, num_classes=total_words)


In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(150))  # Reduced LSTM units
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [ ]:
model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
10/10 [==============================] - 6s 119ms/step - loss: 5.4456
Epoch 2/100
10/10 [==============================] - 1s 123ms/step - loss: 5.4119
Epoch 3/100
10/10 [==============================] - 1s 115ms/step - loss: 5.2653
Epoch 4/100
10/10 [==============================] - 1s 63ms/step - loss: 5.1256
Epoch 5/100
10/10 [==============================] - 1s 61ms/step - loss: 5.0370
Epoch 6/100
10/10 [==============================] - 1s 66ms/step - loss: 4.9062
Epoch 7/100
10/10 [==============================] - 1s 63ms/step - loss: 4.8200
Epoch 8/100
10/10 [==============================] - 1s 60ms/step - loss: 4.7278
Epoch 9/100
10/10 [==============================] - 1s 62ms/step - loss: 4.6386
Epoch 10/100
10/10 [==============================] - 1s 59ms/step - loss: 4.5540
Epoch 11/100
10/10 [==============================] - 1s 60ms/step - loss: 4.4670
Epoch 12/100
10/10 [==============================] - 1s 59ms/step - loss: 4.3523
Epoch 13/100
10/10 [==

In [ ]:
import numpy as np

def generate_text(seed_text, next_words, model, max_sequence_length, tokenizer, temperature=1.0):
    generated_text = seed_text
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted_probabilities = model.predict(token_list, verbose=0)[0]

        # Apply temperature to the predicted probabilities
        predicted_probabilities = np.log(predicted_probabilities) / temperature
        predicted_probabilities = np.exp(predicted_probabilities)
        predicted_probabilities /= np.sum(predicted_probabilities)

        # Sample a word based on the adjusted probabilities
        predicted = np.random.choice(len(predicted_probabilities), size=1, p=predicted_probabilities)[0]

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
        generated_text += " " + output_word
    return generated_text



In [ ]:
seed_text = "Muslims BUSTED: They Stole Millions In Gov’t Benefits"
next_words = 20

generated_text = generate_text(seed_text, next_words, model, max_sequence_length, tokenizer)
print(generated_text)

Muslims BUSTED: They Stole Millions In Gov’t Benefits benefits the daddy vote pedophile least if 6 for sex for of for inappropriate handling information information » 100percentfedup com
